In [7]:
import torch
from torch import nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1 生成数据并显示

In [35]:
path_youwu = "F:\OneDrive\AI资料\机器学习\浩杰给的课件\pytorch\实验数据\去雾数据集\有雾图片"
path_quwu = "F:\OneDrive\AI资料\机器学习\浩杰给的课件\pytorch\实验数据\去雾数据集\去雾图片"

def readImagesToNumpy(path):
    x = []
    for filename in os.listdir(path, ):
        #print(filename)
        file = path + '\\' + filename
        img = Image.open(file)
        img = img.resize((200, 100), Image.ANTIALIAS) #read the image
        img = np.transpose(img, (2, 0, 1))
        img = np.array(img) # transform to numpy
        img = img / 255 # normalize
        #plt.imshow(img)
        #print(img.shape)
        x.append(img)
    return x

X = readImagesToNumpy(path_youwu)
y = readImagesToNumpy(path_quwu)
X = torch.tensor(X, device=device)
y = torch.tensor(y, device=device)

train_num = len(X)*4//5
X_train = X[:train_num, :]
y_train = y[:train_num, :]
X_test = X[train_num:, :]
y_test = y[train_num:, :]

print(X.shape, X.device, y.shape, y.device)

torch.Size([520, 100, 200, 3]) torch.Size([520, 100, 200, 3])


## 2 读取数据

In [ ]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
train_dataset = torch.utils.data.dataset(X_train, y_train, transform=transform)
test_dataset = torch.utils.data.dataset(X_test, y_test, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.Dataloader(test_dataset, batch_size=batch_size, shuffle=True)

for X, y in train_dataset:
    print(X[0, :], y[0, :])
    plt.imshow(Image.fromarray(X[0, :]))
    plt.imshow(Image.fromarray(y[0, :]))
    break

## 3 定义卷积层和模型

In [9]:
def corr2d(X, K):
    batch_size, H, W = X.shape
    h_k, w_k = K.shape
    Y = torch.zeros((batch_size, H - k_h + 1, W - k_w + 1)).to(device)
    for i in range(Y.shape[1]):
        for j in range(Y.shape[2]):
            Y[:, i, j] = (X[:, i : i + k_h, j : j + k_w] * K).sum()
    return Y

def corr2d_multi_in(X, K):
    res = corr2d(X[:, 0, :, :], K[0, :, :])
    for i in range(1, X.shape[1]):
        res += corr2d(X[:, i, :, :], K[i, :, :])
    return res

def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], dim=1)

class MyConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(MyConv2D, self).__init__()
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        self.weight = nn.Parameter(torch.randn((out_channels, in_channels) + kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels, 1, 1))

    def forward(self, x):
        return corr2d_multi_in_out(x, self.weight) + self.bias

In [10]:
class MyConvModule(nn.Module):
    def __init__():
        super(MyConvModule, self).__init__()
        self.conv = nn.Sequential(
            MyCon2D(in_channels=3, out_channels=32, kernel_size=3)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Linear()

SyntaxError: invalid syntax (<ipython-input-10-4d857bae4ba1>, line 6)